In [1]:
import requests
import json


def hlpGeocode(add):
    url = (geocode_url + (
        requests.utils.quote(str(add).strip('\n'))) + '&app_id=' + app_id + '&app_code=' + app_code + '&' + gen)
    json_result = json.loads(requests.get(url, params=place_headers).text + '\n')
    display_location_lat = json_result.get('Response').get('View')[0].get('Result')[0].get('Location').get(
        'DisplayPosition').get('Latitude')
    display_location_lon = json_result.get('Response').get('View')[0].get('Result')[0].get('Location').get(
        'DisplayPosition').get('Longitude')
    print(add, display_location_lat, display_location_lon)
    return '{},{}'.format(str(display_location_lat), str(display_location_lon))


app_id = 'inMaDeyec4Eyevupache'  # YOUR APP ID
app_code = '-T00kPbHqMe2-6e94f-Grw'  # YOUR APP CODE
gen = 'gen=8'
lang = 'en-US'
place_url = 'places.api.here.com'
place_headers = {'Accept-Language': lang}
geocode_url = 'http://geocoder.api.here.com/6.2/geocode.json?searchtext='
wse_entry = 'https://wse.cit.api.here.com/2/findsequence.json?'

destination_list = [0, '台北市中正區北平西路3號', 0],\
                   [1, '臺北市八德路4段692號4樓', 1800],\
                   [2, '臺北市信義路5段15號2樓', 1800],\
                   [3, '臺北市新生南路2段86號', 1800],\
                   [4, '臺北市松江路367號2樓', 1800],\
                   [5, '臺北市忠孝東路1段108號7樓', 1800],\
                   [6, '臺北市昌吉街57號3樓之1', 1800],\
                   [7, '臺北市和平西路3段120號4樓', 1800],\
                   [8, '臺北市木柵路3段220號2樓', 1800],\
                   [9, '臺北市南港路1段360號4樓', 1800],\
                   [10, '臺北市民權東路6段99號3樓', 1800],\
                   [11, '臺北市中正路439號3樓', 1800],\
                   [12, '臺北市新市街30號3樓', 1800],\
                   [13, '台北市士林區華岡路55號', 0]

print('geocoding for waypoints...')

start_url = ''
end_url = ''
dests_url = ''

for dest in destination_list:
    sn, add, stay = dest
    if sn == 0:
        start_url += 'start={};{}&'.format(sn, hlpGeocode(add))
    elif sn == len(destination_list):
        end_url += 'end={};{}'.format(sn, hlpGeocode(add))
    else:
        dests_url += 'destination{}={};{};st:{}&'.format(sn, sn, hlpGeocode(add), stay)

print('geocoding completed!\n')

full_waypoints_url = '{}{}{}'.format(start_url, dests_url, end_url)

departure = '2018-07-15T01:00:00Z'  # UTC
improveFor = 'time'
mode = 'fastest;car;traffic:enabled'
wse_request_url = '{}&{}&improveFor={}&mode={}&departure={}&app_id={}&app_code={}'.format(wse_entry, full_waypoints_url,
                                                                                          improveFor, mode, departure,
                                                                                          app_id, app_code)
print(wse_request_url)
print('\nrequesting WSE...')
wse_result_list = []
wse_result = json.loads(requests.get(wse_request_url).text)
results = wse_result['results']
for result in results:
    waypoints = result['waypoints']
    for waypoint in waypoints:
        wp_id = int(waypoint['id'])
        wp_name = destination_list[wp_id][1]
        wp_lat = waypoint['lat']
        wp_lon = waypoint['lng']
        wp_seq = waypoint['sequence']
        wp_arr = waypoint['estimatedArrival']
        wp_dep = waypoint['estimatedDeparture']
        wp_constraints = waypoint['fulfilledConstraints']
        wse_result_list.append((wp_id, wp_name, wp_lat, wp_lon, wp_seq, wp_arr, wp_dep, wp_constraints))
        print(
            'SEQ={}\tID={}\tNAME={}\tARR={}\tDEP={}'.format(wp_seq, wp_id, destination_list[wp_id][1], wp_arr, wp_dep))


geocoding for waypoints...
台北市中正區北平西路3號 25.04717 121.51706
臺北市八德路4段692號4樓 25.04985 121.57717
臺北市信義路5段15號2樓 25.03314 121.5669
臺北市新生南路2段86號 25.02665 121.53461
臺北市松江路367號2樓 25.06439 121.5335
臺北市忠孝東路1段108號7樓 25.04425 121.52477
臺北市昌吉街57號3樓之1 25.06601 121.51514
臺北市和平西路3段120號4樓 25.035 121.49989
臺北市木柵路3段220號2樓 24.98959 121.57008
臺北市南港路1段360號4樓 25.05453 121.60676
臺北市民權東路6段99號3樓 25.06922 121.58893
臺北市中正路439號3樓 25.09284 121.51969
臺北市新市街30號3樓 25.13269 121.50258
台北市士林區華岡路55號 25.13752 121.54041
geocoding completed!

https://wse.cit.api.here.com/2/findsequence.json?&start=0;25.04717,121.51706&destination1=1;25.04985,121.57717;st:1800&destination2=2;25.03314,121.5669;st:1800&destination3=3;25.02665,121.53461;st:1800&destination4=4;25.06439,121.5335;st:1800&destination5=5;25.04425,121.52477;st:1800&destination6=6;25.06601,121.51514;st:1800&destination7=7;25.035,121.49989;st:1800&destination8=8;24.98959,121.57008;st:1800&destination9=9;25.05453,121.60676;st:1800&destination10=10;25.06922,121.58893;st:18

In [2]:
import folium
from random import randint
from math import radians, cos, sin, degrees, atan2, atan, tan, acos

def get_degree(latA, lonA, latB, lonB):
    radLatA = radians(latA)
    radLonA = radians(lonA)
    radLatB = radians(latB)
    radLonB = radians(lonB)
    dLon = radLonB - radLonA
    y = sin(dLon) * cos(radLatB)
    x = cos(radLatA) * sin(radLatB) - sin(radLatA) * cos(radLatB) * cos(dLon)
    brng = degrees(atan2(y, x))
    brng = (brng + 360) % 360
    return brng

def get_route(ori_lat, ori_lon, dest_lat, dest_lon, route_mode, departure_time):
    route_url = 'https://route.api.here.com/routing/7.2/calculateroute.json?'
    wp0 = ('{},{}'.format(ori_lat, ori_lon))
    wp1 = ('{},{}'.format(dest_lat, dest_lon))
    route_options = '&mode={}&departure={}&legAttributes=shape'.format(mode, departure_time)
    url = route_url + 'app_id=' + app_id + '&app_code=' + app_code + '&waypoint0=geo!' + wp0 + '&waypoint1=geo!' + wp1 + route_options
    json_result = json.loads(requests.get(url).text)
    routes = json_result['response']['route']
    route_shape = []
    for route in routes:
        legs = route['leg']
        for leg in legs:
            shape = leg['shape']
            point_index = 0
            while point_index < len(shape):
                point = shape[point_index]
                lat = float(point.split(',')[0])
                lon = float(point.split(',')[1])
                if point_index < len(shape) - 1:
                    avg_lat = (float(lat) + float(shape[point_index + 1].split(',')[0])) / 2
                    avg_lon = (float(lon) + float(shape[point_index + 1].split(',')[1])) / 2
                    bearing = get_degree(lat, lon, float(shape[point_index + 1].split(',')[0]), float(shape[point_index + 1].split(',')[1]))
                route_shape.append(([lat,lon], [avg_lat, avg_lon], bearing))
                point_index += 1
    return route_shape

m = folium.Map(
    location = [23, 121],
    tiles = 'https://1.base.maps.cit.api.here.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?lg=cht&&app_id=' + app_id + '&app_code=' + app_code,
    #detect_retina = True,
    max_zoom = 20,
    attr =  '(c)1987-2018 HERE'
)

bounds = []

i = 0
while i < len(wse_result_list):
    rand_color = '#'
    j = 0
    while j < 6:
        rand_color += hex(randint(5,16))[-1]
        j += 1
    wp_id, wp_name, wp_lat, wp_lon, wp_seq, wp_arr, wp_dep, wp_constraints = wse_result_list[i]
    if i == 0 or i == len(wse_result_list) - 1:
        icon = folium.map.Icon(color='red', icon_color='white')#, icon='info-sign', angle=0, prefix='glyphicon')
    else:
        icon = folium.map.Icon(color='blue', icon_color='white')#, icon='info-sign', angle=0, prefix='glyphicon')
    folium.Marker([wp_lat, wp_lon], popup='{}<br>{}<br>抵達時間：{}<br>離開時間：{}'.format(wp_id, wp_name, wp_arr, wp_dep), icon=icon).add_to(m)
    bounds.append([wp_lat, wp_lon])
    shape_point_list = []
    if i < len(wse_result_list) - 1:
        print('calculating route for waypoint {} --> {} / {} --> {}'.format(wp_seq, wse_result_list[i + 1][4], wp_name, wse_result_list[i + 1][1]))
        route_shape = get_route(wp_lat, wp_lon, wse_result_list[i + 1][2], wse_result_list[i + 1][3], mode, wp_dep)
        shape_point_index = 0
        while shape_point_index < len(route_shape):
            shape_point = route_shape[shape_point_index]
            shape_point_list.append(shape_point[0])
            if len(route_shape) > 10 and shape_point_index % 10 == 0:
                folium.features.RegularPolygonMarker(shape_point[1], color=rand_color, opacity=1, weight=2, fill_color=rand_color, fill_opacity=1, number_of_sides=3, rotation=int(shape_point[2])-90, radius=7).add_to(m)
            shape_point_index += 1
        folium.PolyLine(shape_point_list, color=rand_color, weight=4, opacity=1).add_to(m)
    i += 1

m.fit_bounds(bounds)
m

calculating route for waypoint 0 --> 1 / 台北市中正區北平西路3號 --> 臺北市和平西路3段120號4樓
calculating route for waypoint 1 --> 2 / 臺北市和平西路3段120號4樓 --> 臺北市忠孝東路1段108號7樓
calculating route for waypoint 2 --> 3 / 臺北市忠孝東路1段108號7樓 --> 臺北市新生南路2段86號
calculating route for waypoint 3 --> 4 / 臺北市新生南路2段86號 --> 臺北市木柵路3段220號2樓
calculating route for waypoint 4 --> 5 / 臺北市木柵路3段220號2樓 --> 臺北市信義路5段15號2樓
calculating route for waypoint 5 --> 6 / 臺北市信義路5段15號2樓 --> 臺北市八德路4段692號4樓
calculating route for waypoint 6 --> 7 / 臺北市八德路4段692號4樓 --> 臺北市南港路1段360號4樓
calculating route for waypoint 7 --> 8 / 臺北市南港路1段360號4樓 --> 臺北市民權東路6段99號3樓
calculating route for waypoint 8 --> 9 / 臺北市民權東路6段99號3樓 --> 臺北市松江路367號2樓
calculating route for waypoint 9 --> 10 / 臺北市松江路367號2樓 --> 臺北市昌吉街57號3樓之1
calculating route for waypoint 10 --> 11 / 臺北市昌吉街57號3樓之1 --> 臺北市中正路439號3樓
calculating route for waypoint 11 --> 12 / 臺北市中正路439號3樓 --> 臺北市新市街30號3樓
calculating route for waypoint 12 --> 13 / 臺北市新市街30號3樓 --> 台北市士林區華岡路55號
